In [1]:
!pip install opencv-python tensorflow scikit-learn matplotlib

In [2]:
import os
import zipfile
import shutil
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam


In [3]:
with zipfile.ZipFile("/content/images.zip", "r") as zip_ref:
    zip_ref.extractall("dataset")

FileNotFoundError: [Errno 2] No such file or directory: '/content/images.zip'

In [ ]:
EMOTIONS = [
    "Anger",
    "Contempt",
    "Disgust",
    "Fear",
    "Happy",
    "Sad",
    "Surprised",
    "Neutral"
]

emotion_to_label = {e: i for i, e in enumerate(EMOTIONS)}
label_to_emotion = {i: e for e, i in emotion_to_label.items()}


In [ ]:
IMG_SIZE = 48

X = []
y = []

base_path = "/content/dataset/images"

for subject_id in os.listdir(base_path):  # 0 → 10
    subject_path = os.path.join(base_path, subject_id)
    if not os.path.isdir(subject_path):
        continue

    for emotion in EMOTIONS:
        img_path = os.path.join(subject_path, f"{emotion}.jpg")

        if not os.path.exists(img_path):
            continue

        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        if img is None:
            continue

        img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
        X.append(img)
        y.append(emotion_to_label[emotion])


In [ ]:
X = np.array(X, dtype="float32") / 255.0
X = X.reshape(-1, IMG_SIZE, IMG_SIZE, 1)
y = to_categorical(y, num_classes=len(EMOTIONS))

print("Total images loaded:", X.shape[0])


In [ ]:
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y,
    test_size=0.30,
    random_state=42,
    stratify=y
)

X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp,
    test_size=0.50,
    random_state=42,
    stratify=y_temp
)

print("Train:", X_train.shape)
print("Validation:", X_val.shape)
print("Test:", X_test.shape)


In [ ]:
model = Sequential([
    Conv2D(16, (3,3), activation='relu', input_shape=(48,48,1)),
    MaxPooling2D(2,2),

    Conv2D(32, (3,3), activation='relu'),
    MaxPooling2D(2,2),

    Flatten(),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(len(EMOTIONS), activation='softmax')
])

model.compile(
    optimizer=Adam(learning_rate=0.0005),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()


In [ ]:
history = model.fit(
    X_train, y_train,
    epochs=40,
    batch_size=8,
    validation_data=(X_val, y_val)
)


In [ ]:
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_acc:.2f}")


In [ ]:
plt.figure(figsize=(12,4))

plt.subplot(1,2,1)
plt.plot(history.history['accuracy'], label='Train')
plt.plot(history.history['val_accuracy'], label='Val')
plt.title("Accuracy")
plt.legend()

plt.subplot(1,2,2)
plt.plot(history.history['loss'], label='Train')
plt.plot(history.history['val_loss'], label='Val')
plt.title("Loss")
plt.legend()

plt.show()


In [ ]:
def predict_emotion(image_path):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (48,48))
    img = img / 255.0
    img = img.reshape(1,48,48,1)

    preds = model.predict(img)
    emotion = label_to_emotion[np.argmax(preds)]
    confidence = np.max(preds)

    return emotion, confidence


In [ ]:
emotion, confidence = predict_emotion("/content/dataset/images/0/Happy.jpg")
print("Emotion:", emotion)
print("Confidence:", confidence)


In [ ]:
model.save("emotion_cnn_small_dataset.h5")
